In [1]:
import pandas as pd
import numpy as np
import math

Boletim Macros: https://coronavirus.saude.mg.gov.br/boletim-especial-macrorregioes

Obtendo o Banco de Dados da COVID-19 no Brasil com informações de número de casos e mortos em cada cidade.

In [4]:
dadosCovid = pd.read_csv("caso_full.csv.gz",index_col=False,header=None,squeeze=True,low_memory=False)
#Origem dos dados: https://brasil.io/dataset/covid19/files/

In [5]:
display(dadosCovid.loc[[0]])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths


Reduzindo os dados para as cidades de Minas Gerais apenas.

In [6]:
dadosMG = dadosCovid[dadosCovid[15] == "MG"]
dadosMG = dadosMG.dropna()
dadosMG.columns
#Convertendo para número algumas colunas 
dadosMG[16] = pd.to_numeric(dadosMG[16])
dadosMG[17] = pd.to_numeric(dadosMG[17])
#Fonte: https://www.ti-enxame.com/pt/python/alterar-tipo-de-dados-de-colunas-em-pandas/1072113742/

Escolhendo uma cidade específica para conhecer melhor algumas informações do dataset.

In [11]:
cidade = "Florestal"
dadosCidade = dadosMG[dadosMG[0] == cidade]
display(dadosCidade)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
638502,Florestal,3126000,2020-05-25,202022,7533,7461,False,False,1,13.27492,2020-05-25,0.0000,0,1,city,MG,1,0
638905,Florestal,3126000,2020-05-26,202022,7533,7461,False,False,1,13.27492,2020-05-26,0.0000,0,2,city,MG,0,0
639318,Florestal,3126000,2020-05-27,202022,7533,7461,False,False,1,13.27492,2020-05-27,0.0000,0,3,city,MG,0,0
639744,Florestal,3126000,2020-05-28,202022,7533,7461,False,False,2,26.54985,2020-05-28,0.0000,0,4,city,MG,1,0
640182,Florestal,3126000,2020-05-29,202022,7533,7461,False,False,2,26.54985,2020-05-29,0.0000,0,5,city,MG,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001165,Florestal,3126000,2021-08-04,202131,7533,7461,False,True,288,3823.17802,2021-06-18,0.0417,12,437,city,MG,0,0
1002020,Florestal,3126000,2021-08-05,202131,7533,7461,False,True,288,3823.17802,2021-06-18,0.0417,12,438,city,MG,0,0
1002875,Florestal,3126000,2021-08-06,202131,7533,7461,False,True,288,3823.17802,2021-06-18,0.0417,12,439,city,MG,0,0
1003730,Florestal,3126000,2021-08-07,202131,7533,7461,False,True,288,3823.17802,2021-06-18,0.0417,12,440,city,MG,0,0


In [8]:
TotalCasos = dadosCidade[16].sum()
TotalObitos = dadosCidade[17].sum()
#Fonte: https://www.delftstack.com/pt/howto/python-pandas/how-to-get-the-sum-of-pandas-column/
print(f"Total de casos em {cidade} é {TotalCasos}")
print(f"Total de óbitos em {cidade} é {TotalObitos}")

Total de casos em Florestal é 288
Total de óbitos em Florestal é 12


In [4]:
#Subset das informações mais relevantes sobre a vacinação.
#Chunks: Ler pequenos subsets de dados: https://www.ti-enxame.com/pt/python/como-ler-um-arquivo-csv-de-6-gb-com-pandas/1049682858/
dadosVacinacao = pd.read_csv("vacinacao.zip",usecols=['ID','faixa_etaria','sexo','municipio_res','macro','vacina_nome','data_aplicacao','dose'])
#Fonte: https://coronavirus.saude.mg.gov.br/dadosabertos -  Sistemas Vacinação

In [5]:
display(dadosVacinacao)

,ID,faixa_etaria,sexo,municipio_res,macro,vacina_nome,data_aplicacao,dose
0,1,60 a 69 anos,MASCULINO,BELO HORIZONTE,CENTRO,CORONAVAC,2021-05-20,2ª Dose
1,2,70 a 79 anos,FEMININO,SANTA LUZIA,CENTRO,CORONAVAC,2021-03-17,1ª Dose
2,3,70 a 79 anos,FEMININO,PORTEIRINHA,NORTE,CORONAVAC,2021-04-26,2ª Dose
3,4,60 a 69 anos,FEMININO,IPATINGA,VALE DO ACO,ASTRAZENECA,2021-04-22,1ª Dose
4,5,70 a 79 anos,MASCULINO,UBERLANDIA,TRIANGULO DO NORTE,CORONAVAC,2021-04-01,2ª Dose
...,...,...,...,...,...,...,...,...
13248422,13248423,40 a 49 anos,MASCULINO,BELO HORIZONTE,CENTRO,PFIZER,2021-07-02,1ª Dose
13248423,13248424,70 a 79 anos,FEMININO,SANTA RITA DO SAPUCAI,SUL,CORONAVAC,2021-03-27,1ª Dose
13248424,13248425,40 a 49 anos,FEMININO,BELO HORIZONTE,CENTRO,ASTRAZENECA,2021-07-05,1ª Dose
13248425,13248426,50 a 59 anos,FEMININO,MATIAS BARBOSA,SUDESTE,ASTRAZENECA,2021-05-12,1ª Dose
